In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from datetime import date, timedelta
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException



парсим ВСЕ c ОКПД2 = 62,63. Это не совсем конструктивно, дабы нудно избавиться от кучи чуши связаной с закупкой ПО и прочим нерелевантным бредом) лечить будем путем более тщательного выбора ОКПД2 и замены ссылки (либо подстановки в ссылку значений)

In [10]:
publish_date = date.today() - timedelta(days=1)
# подставляем дату вчерашнего дня (пока не уточним какую надо точно)
formatted_date = publish_date.strftime("%d-%m-%Y")
option = Options()
option.add_argument("--disable-infobars")
s=Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=s, options=option)
browser.get('https://zakupki.gov.ru/epz/order/extendedsearch/results.html?morphology=on \
&search-filter=%D0%94%D0%B0%D1%82%D0%B5+%D1%80%D0%B0%D0%B7%D0%BC%D0%B5%D1%89%D0%B5%D0%BD%D0%B8%D1%8F&pageNumber=1&sortDirection=false \
    &recordsPerPage=_50&showLotsInfoHidden=false&savedSearchSettingsIdHidden=++++++++setting_order_lrw8g1vl&sortBy=UPDATE_DATE \
        &fz44=on&fz223=on&af=on&ca=on&selectedLaws=FZ44%2CFZ223&currencyIdGeneral=-1&publishDateFrom='+str(formatted_date)+'&okpd2IdsWithNested=on&okpd2Ids=8873937%2C8873938 \
            &okpd2IdsCodes=62%2C63&gws=%D0%92%D1%8B%D0%B1%D0%B5%D1%80%D0%B8%D1%82%D0%B5+%D1%82%D0%B8%D0%BF+%D0%B7%D0%B0%D0%BA%D1%83%D0%BF%D0%BA%D0%B8 \
                &OrderPlacementSmallBusinessSubject=on&OrderPlacementRnpData=on&OrderPlacementExecutionRequirement=on&orderPlacement94_0=0&orderPlacement94_1=0&orderPlacement94_2=0')

try:
    # Ожидаем появления модального окна с проклятым регионом
    WebDriverWait(browser, 15).until(EC.presence_of_element_located((By.ID, 'region-select')))

    # Найдите кнопку закрытия и нажмите на нее
    close_button = browser.find_element(By.CSS_SELECTOR, ".btn-close.closePopUp")
    close_button.click()

except TimeoutException:
    # Если модальное окно не появилось в течение 10 секунд, просто продолжаем
    pass


time.sleep(2)
share =  browser.find_element(By.CLASS_NAME,  'downLoad-search' ) #тут мы ицщем кнопку "выгрузить результаты поиска"
share.click() #жмякаем ее
time.sleep(2)
checkbox = browser.find_element(By.XPATH, '//span[@class="dynatree-title" and text()="Все параметры"]/preceding-sibling::span[@class="dynatree-checkbox"]')
checkbox.click() #ставим чекбокс "выбрать все" в окне параметров загрузки 
button =  browser.find_element(By.ID,  'btn-primary' ).click() #жмем далее
time.sleep(2) 

#тут мы жмем на ссылочки для скачивания csv по 500 резултатов поиска. их не больше 10 - поэтому такой цикл
for i in range(1, 11):
    xpath = f'/html/body/div[3]/div/div/div[3]/div[4]/div[2]/div/div[1]/div[{i}]/div/div[2]'
    elements = browser.find_elements(By.XPATH, xpath)
    for element in elements:
        element.click()
        time.sleep(5) # Пауза



browser.implicitly_wait(20)
browser.quit()